# 1. 開啟瀏覽器，進到FB首頁

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd


# 關閉通知
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values':
        {
            'notifications': 2
        }
}
options.add_experimental_option('prefs', prefs)
options.add_argument("disable-infobars")

In [2]:
# 打啟動selenium 務必確認driver 檔案跟python 檔案要在同個資料夾中
# 在這邊下載: https://chromedriver.chromium.org/downloads
driver = webdriver.Chrome(options=options)
driver.get("https://www.facebook.com/")
time.sleep(4)

In [3]:
email = "your_email"
password = "your_password"

#輸入 email 
context = driver.find_element(By.ID, 'email')
context.send_keys(email)
time.sleep(0.5)

#輸入 password
context = driver.find_element(By.ID, 'pass')
context.send_keys(password)
time.sleep(0.5)

# 登入
commit = driver.find_element(By.NAME, 'login')
commit.click()
time.sleep(7)
print("已登入")

已登入


# 2. 進入台大交流版

## 2.1 手動選擇「最新貼文」

In [ ]:
# 進入交流版
driver.get("https://www.facebook.com/groups/NTU.Head")

In [ ]:
# 選擇最新貼文 (預設是熱門貼文)
# 點開選項
commit = driver.find_element(By.XPATH, "//div[@class='j83agx80']//span[@class='a8c37x1j ni8dbmo4 stjgntxs l9j0dhe7']")
commit.click()

# 三個選項，依序是「最新動態」、「最近貼文」、「最相關貼文」，選第二個
commit = driver.find_elements(By.XPATH, "//div[@class='bp9cbjyn j83agx80 btwxx1t3 buofh1pr i1fnvgqd hpfvmrgz']")
commit[1].click()

## 2.2 自動以「最新貼文」排序

In [4]:
# 進入交流版 (以最新貼文排序) (取代上兩格)
driver.get("https://www.facebook.com/groups/NTU.Head?sorting_setting=CHRONOLOGICAL")

# 3. 展開頁面

## 3.1 往下滑以顯示更多篇文章

In [5]:
# 往下滑
for i in range(0,5):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)
print("滑好了")

滑好了


## 3.2 點開「顯示更多」

In [13]:
# 點「顯示更多」 (要設兩層條件，因為貼文者那邊 其他6人也是div後面這個class) (要多執行幾次 直到數字不便)

commit = driver.find_elements(By.XPATH, "//div[@class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gpro0wi8 oo9gr5id lrazzd5p' and @role='button']")
print(len(commit))

for c in commit:
    try:
        c.click()
    except:
        print(c)

commit = driver.find_elements(By.XPATH, "//div[@class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gpro0wi8 oo9gr5id lrazzd5p' and @role='button']")
if len(commit) == 0:
    print("都點開了")
else:
    print("點開了")

4
<selenium.webdriver.remote.webelement.WebElement (session="23b9c06a57b0e3608e8059b7fecd2c9f", element="e25aea22-ff13-454c-835e-0b98225c965c")>
<selenium.webdriver.remote.webelement.WebElement (session="23b9c06a57b0e3608e8059b7fecd2c9f", element="f901ffd6-52be-40da-b92c-96bd0c482194")>
<selenium.webdriver.remote.webelement.WebElement (session="23b9c06a57b0e3608e8059b7fecd2c9f", element="5b5f5c5f-7203-4f2c-9fd3-e6d5abf7aaf9")>
點開了


## 3.3 將游標移至「時間」上，以方便後續獲取貼文連結

In [14]:
urls = driver.find_elements(By.XPATH, "//div[@class='du4w35lb k4urcfbm l9j0dhe7 sjgh65i0']//span[@class='tojvnm2t a6sixzi8 abs2jz4q a8s20v7p t1p8iaqh k5wvi7nf q3lfd5jv pk4s997a bipmatt0 cebpdrjk qowsmv63 owwhemhu dp1hu0rb dhp61c6y iyyx5f41']//a[@role='link']")
print(len(urls))

action = ActionChains(driver)
for url in urls:
    action.move_to_element(url).perform()
print("OK")

26
OK


# 4. 擷取網頁資料

## 4.1 擷取並解析 HTML

In [25]:
html = etree.HTML(driver.page_source)
soup = BeautifulSoup(driver.page_source, "html.parser")

## 4.2 選擇貼文區

In [26]:
# 貼文區是 role=feed，只會有一個
all_ = soup.find_all("div", role="feed")
print(len(all_))

1


## 4.3 獲取文章資訊

### 4.3.1 以一篇文章為單位，獲取「作者」、「時間」、「連結」、「內容」

In [28]:
# 一篇PO文
# div class_="du4w35lb k4urcfbm l9j0dhe7 sjgh65i0"
# 文字的部分
# div class_="ecm0bbzt hv4rvrfc ihqw7lf3 dati1w0a" (比"qzhwtbm6 knvmm38d"層級高)
# div class_="dati1w0a ihqw7lf3 hv4rvrfc ecm0bbzt"
# 每篇PO文的重要訊息們 (一般是3個)
# div class_="qzhwtbm6 knvmm38d"


all_2 = all_[0].find_all("div", class_="du4w35lb k4urcfbm l9j0dhe7 sjgh65i0")
print(len(all_2))

name_list = []
time_list = []
url_list = []
content_list = []

for i in range(len(all_2)):
    # inform 前2個固定是名字和時間，但後面數量不一定，依貼文形式而定
    inform = all_2[i].find_all("div", class_="qzhwtbm6 knvmm38d")
    print(i)
    #print("inform", len(inform))

    
    # 名字 name
    n = inform[0].find_all("h2")
    name = n[0].find("span").text
    print(name)
    name_list.append(name)

    # 時間 time
    t = inform[1].find_all("a", class_ = "oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw")
    time = t[0].find("span").text.strip("=")
    print(time)
    time_list.append(time)
    
    # 文章連結 url (滑鼠游標移到上面之後 HTML裡才會有，不然都是#)
    url = t[0].get("href")
    print(url)
    url_list.append(url)
    
    # 內容 content
    try:
        c = inform[2].find_all("div", dir="auto")
        
        # 如果貼文的是一群人分享文章就會進到這邊
        if len(c) == 0:
            words = all_2[i].find_all("div", class_="dati1w0a ihqw7lf3 hv4rvrfc ecm0bbzt")
            w = words[0]
            # print(w.text)
            content_list.append(w.text)
        
        # 一般文章走這邊
        else:
            txt = ""
            for t in c:
                # print(t.text)
                txt += t.text     # 把不同行的內容結合在一起
            content_list.append(txt)

    except:
        # 如果貼文的是一群人單純 PO 文章就會進到這邊
        try:
            words = all_2[i].find_all("div", class_="dati1w0a ihqw7lf3 hv4rvrfc ecm0bbzt")
            w = words[0]
            # print(w.text)
            content_list.append(w.text)
        
        except:
            print("Ooops")
            content_list.append("No content")

    print("\n")

print(len(name_list))
print(len(time_list))
print(len(url_list))
print(len(content_list))

27
0
黃鈺紜
8分鐘
https://www.facebook.com/groups/NTU.Head/posts/1948687685336017/?__cft__[0]=AZWqbE0xxV0XDd1u3C7N0cpisClgs1DkBscsxVzG5PpDRjfpJodeQhEwH8oLQ7XUpOe6H6sB17nxVe1DZPyIgtQQ3PIGKREMAoIq3OHu6ndz1WkoT0DqkZWHCtx4XcCoCKkXYVszCkaxbJf0GHIEYMdyGYDAJfJN-Ga78G2aOXlDUMY4-PWEG_L7COhpcUKrzV8&__tn__=%2CO%2CP-R


1
李欣芮
13分鐘
https://www.facebook.com/groups/NTU.Head/posts/1948684235336362/?__cft__[0]=AZWCKDQzjepAhVvDNulq0NoBlCb2BF_u515d_sHDUyyu_pAOiXj_5b0YWP_Ei3fiNNHAYyva_CB5eTaDxX4hNPBA4qTMR42rnomkz9A_E8Ah34LaOlq7vdlg8CarPhmKcXAOiVhzv541Mq8DM5VFcvwSlTYiDYC4IwcgENgpb9OMKb9vlty1dsE3CIAdm7X6mVo&__tn__=%2CO%2CP-R


2
Rohit Das
14分鐘
https://www.facebook.com/groups/NTU.Head/posts/1948682365336549/?__cft__[0]=AZUhW-Cdm3rMJJEcbz-adGs6ZYgFNdafoR3uEp0CeRFhus6XVjRnjqK51lpjziGJs_vGrPyvqXO6ShxPU6esrefvs0Prbk1ncLuupFaKm6wOxOihGaHlYNVOs3a_mz32vPYf-GmJaXQLLa3F6y0QY2qNyihPknGE5pv5sjkrmudMEZ56ibwLrgYPMt0yRMn0kQxg-lnTBtXGhe7g4g_M70Mz&__tn__=%2CO%2CP-R


3
謝恩慈
1小時
https://www.facebook.com/groups/NTU.Head/posts/19486

### 4.3.2 列印出前3組資料

In [22]:
for i in range(3):
    print(name_list[i])
    print(time_list[i])
    print(url_list[i])
    print(content_list[i])
    print("\n")

黃鈺紜
5分鐘
https://www.facebook.com/groups/NTU.Head/posts/1948687685336017/?__cft__[0]=AZWqbE0xxV0XDd1u3C7N0cpisClgs1DkBscsxVzG5PpDRjfpJodeQhEwH8oLQ7XUpOe6H6sB17nxVe1DZPyIgtQQ3PIGKREMAoIq3OHu6ndz1WkoT0DqkZWHCtx4XcCoCKkXYVszCkaxbJf0GHIEYMdyGYDAJfJN-Ga78G2aOXlDUMY4-PWEG_L7COhpcUKrzV8&__tn__=%2CO%2CP-R
尋找藥袋⋯⋯今天晚上七點多把藥忘在鹿鳴堂對面Ubike站的車籃了，希望如果有人看到能聯絡我！或隨便丟在旁邊⋯⋯我再自己拿（或是我自體痊癒）非常感謝⋯⋯（順道一提，我騎這台Ubike的時候撞到一隻大勞贖）


李欣芮
10分鐘
https://www.facebook.com/groups/NTU.Head/posts/1948684235336362/?__cft__[0]=AZWCKDQzjepAhVvDNulq0NoBlCb2BF_u515d_sHDUyyu_pAOiXj_5b0YWP_Ei3fiNNHAYyva_CB5eTaDxX4hNPBA4qTMR42rnomkz9A_E8Ah34LaOlq7vdlg8CarPhmKcXAOiVhzv541Mq8DM5VFcvwSlTYiDYC4IwcgENgpb9OMKb9vlty1dsE3CIAdm7X6mVo&__tn__=%2CO%2CP-R
【新體需要你 填問卷抽300元️】———————————————————————問卷連結： https://docs.google.com/....../1UBU9XIptM....../edit———————————————————————交流版的各位晚安我們是修習許嘉文老師開設的管理學的BMP專案小組！場地租借時，負責人和櫃檯阿姨總是說法不一嗎？健身房設備老舊、總是人擠人嗎？每週早八早起搶羽球場，卻因為系統操作不便而功虧一簣嗎？新體知道你的困擾，希望能聽到大家的聲音！相信版上有許多經常在新體運動的朋友們！我們發現新體在場地租借等方面有些讓使用者不方便的地方，並在與新體的老師們訪談後，了解

### 4.3.3 檢查社團內PO文是否有關鍵字

In [29]:
keywords = ["免費", "便當", "剩下", "飲料"]
count = 0

# 依序添加「姓名」「時間」
dict = {"name":name_list, "time":time_list}

# 若「內容」含有關鍵字，則打勾，反之則留空 (不含分享的文章內容)
for keyword in keywords:
    has_keyword = []
    for i in range(len(content_list)):
        if keyword in content_list[i]:
            count += 1
            has_keyword.append("V")
        else:
            has_keyword.append(" ")
    dict[keyword] = has_keyword

# 把「內容」、「連結」加在最後
dict["content"] = content_list
dict["url"] = url_list

print("在 " +str(len(content_list))+" 篇文中")
print("共發現 "+str(count)+" 篇文中含有關鍵字")
print("關鍵字:", keywords)

data = pd.DataFrame(dict)

在 27 篇文中
共發現 1 篇文中含有關鍵字
關鍵字: ['免費', '便當', '剩下', '飲料']


In [30]:
data

,name,time,免費,便當,剩下,飲料,content,url
0,黃鈺紜,8分鐘,,,,,尋找藥袋⋯⋯今天晚上七點多把藥忘在鹿鳴堂對面Ubike站的車籃了，希望如果有人看到能聯絡我！...,https://www.facebook.com/groups/NTU.Head/posts...
1,李欣芮,13分鐘,,,,,【新體需要你 填問卷抽300元️】———————————————————————問卷連結： ...,https://www.facebook.com/groups/NTU.Head/posts...
2,Rohit Das,14分鐘,,,,,If anybody interested ping me!,https://www.facebook.com/groups/NTU.Head/posts...
3,謝恩慈,1小時,,,,,#一張A3彩色印刷多少錢合理敝人今天去台大附近的印刷店印兩張活動需要的圖案(非海報or特殊紙...,https://www.facebook.com/groups/NTU.Head/posts...
4,康學容,1 小時,,,,,大一女旁cocoCoCo壹番屋門口地上撿到一把鑰匙，我放到柱子旁邊的木板上！很小把感覺是腳踏...,https://www.facebook.com/groups/NTU.Head/posts...
5,陳起軒,1 小時,,,,,【文末抽獎】抽荼蘼咖啡聖誕禮盒符合以下條件 請不要錯過這篇文❶喜歡喝咖啡❷正在挑聖誕節交換禮...,https://www.facebook.com/groups/NTU.Head/posts...
6,龔柏森,1 小時,,,,,[轉貼:校友新創分享+NFT抽獎(現值約1乙太幣)]KryptoGO是利用AI技術提供金融科...,https://www.facebook.com/groups/NTU.Head/posts...
7,Tong Chen,2 小時,,,,,No content,https://www.facebook.com/groups/NTU.Head/posts...
8,郭睿翔,2 小時,,,,,「你們瘋了喔？」「怎樣？」「做個期末報告要送兩千出去？」「我們只是想要看到大家滿滿的笑容，跟...,https://www.facebook.com/groups/NTU.Head/posts...
9,李宥辰,2 小時,,,,,哇，這次除了代幣外，還有不少周邊商品耶,https://www.facebook.com/groups/NTU.Head/posts...


In [31]:
# 建立一個清單，含有各個關鍵字的清單
data_list = [data.loc[data[keyword] == "V"] for keyword in keywords]

all_data = pd.concat(data_list)
all_data = all_data.sort_index()    # 這樣就會依照時間排序
all_data

,name,time,免費,便當,剩下,飲料,content,url
19,許鏵方,9 小時,,,,V,住在大一女A棟五樓的朋朋可以看一下有沒有收錯衣服嗎><我大概一個禮拜前晾衣服發現有一件跟我一...,https://www.facebook.com/groups/NTU.Head/posts...
